# Pretraining GPT2

We shorten the context length because our data set doesn't have enough tokens for the original 1024 context.

In [ ]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,   # Vocabulary size
    "context_length": 256, # Shortened context length (orig: 1024)
    "emb_dim": 768,        # Embedding dimension
    "n_heads": 12,         # Number of attention heads
    "n_layers": 12,        # Number of layers
    "drop_rate": 0.0,      # Dropout rate
    "qkv_bias": False      # Query-key-value bias
}

## Torch Implementation

Import the libraries, and now we have separated the `GPTModel` implementation to a new file to make the notebook smaller, of course.

In [ ]:
import torch
import tiktoken
from scripts.gpt2_model import GPTModel

torch.manual_seed(123)

We will just instantiate the model. Since our config already has 0 drop out, we don't need to turn on `eval` mode for the model.

In [ ]:
model = GPTModel(GPT_CONFIG_124M)

Define some helpers. 

In [ ]:
from scripts.generate import generate_text_simple

def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # add batch dimension
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0) # remove batch dimension
    return tokenizer.decode(flat.tolist())


Let's test the model at the moment. Again we expect it to generate some garbage.

In [ ]:
start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")

token_ids = generate_text_simple(
  model=model,
  idx=text_to_token_ids(start_context, tokenizer),
  max_new_tokens=10,
  context_size=GPT_CONFIG_124M["context_length"]
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

## Loss Calculation

we have 2 batches here, so we add the corresponding "next" token into the targets while maintaining a consistent context length. this gives a "sliding window" effect.

In [ ]:
inputs = torch.tensor([[16833, 3626, 6100],   # ["every effort moves",
                       [40,    1107, 588]])   #  "I really like"]

targets = torch.tensor([[3626, 6100, 345  ],  # [" effort moves you",
                        [1107,  588, 11311]]) #  " really like chocolate"]

inputs, targets

In [ ]:
torch.manual_seed(123)

logits = model(inputs)

probas = torch.softmax(logits, dim=-1) # Probability of each token in vocabulary
print(probas.shape) # Shape: (batch_size, num_tokens, vocab_size)


Probas contains for each token, there are vocab_size probabilities in the last dimension. We want to select the token ID (the index) of the highest number

In [ ]:

probas

For each word, we find an individual token to succeed the text

In [ ]:
token_ids = torch.argmax(probas, dim=-1, keepdim=True)
print("Token IDs:\n", token_ids)

But too bad! we're nowhere close to our target!

In [ ]:
print(f"Targets batch 1: {token_ids_to_text(targets[0], tokenizer)}")
print(f"Outputs batch 1: {token_ids_to_text(token_ids[0].flatten(), tokenizer)}")

print(f"Targets batch 2: {token_ids_to_text(targets[1], tokenizer)}")
print(f"Outputs batch 2: {token_ids_to_text(token_ids[1].flatten(), tokenizer)}")

So what did this probabilities look like for the _real_ indices for targets? You'll find them to be small. The goal now is to increase these probabilities.

In [ ]:
text_idx = 0
target_probas_1 = probas[text_idx, [0, 1, 2], targets[text_idx]]
print("Text 1:", target_probas_1)

text_idx = 1
target_probas_2 = probas[text_idx, [0, 1, 2], targets[text_idx]]
print("Text 2:", target_probas_2)

Take the log for individual probabilities and then concatenate them.

In [ ]:
# Compute logarithm of all token probabilities
log_probas = torch.log(torch.cat((target_probas_1, target_probas_2)))
print(log_probas)

Take the average and negate to make it positive.

In [ ]:
# Calculate the average probability for each token
avg_log_probas = torch.mean(log_probas)
print(avg_log_probas)

In [ ]:
neg_avg_log_probas = avg_log_probas * -1
print(neg_avg_log_probas)


The above is basically what we call "cross entropy loss"

## Cross Entropy

Lets just use torch cross_entropy to do everything we did above.

In [ ]:
# Logits have shape (batch_size, num_tokens, vocab_size)
print("Logits shape:", logits.shape)

# Targets have shape (batch_size, num_tokens)
print("Targets shape:", targets.shape)

In [ ]:
logits_flat = logits.flatten(0, 1)
targets_flat = targets.flatten()

print("Flattened logits:", logits_flat.shape)
print("Flattened targets:", targets_flat.shape)

In [ ]:
loss = torch.nn.functional.cross_entropy(logits_flat, targets_flat)
print(loss)


We can calculate the perplexity too.

In [ ]:
perplexity = torch.exp(loss)
print(perplexity)

## Data Stuff

In [ ]:
with open("data/the-verdict.txt", "r") as f:
  text_data = f.read()

text_data[:99]

In [ ]:
total_characters = len(text_data)
total_tokens = len(tokenizer.encode(text_data))

print("Characters:", total_characters)
print("Tokens:", total_tokens)

In [ ]:
from scripts.prepare_data import create_dataloader_v1

# Train/validation ratio
train_ratio = 0.90
split_idx = int(train_ratio * len(text_data))
train_data = text_data[:split_idx]
val_data = text_data[split_idx:]

torch.manual_seed(123)

train_loader = create_dataloader_v1(
    train_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=True,
    shuffle=True,
    num_workers=0
)

val_loader = create_dataloader_v1(
    val_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=False,
    shuffle=False,
    num_workers=0
)

In [ ]:
# Sanity check

if total_tokens * (train_ratio) < GPT_CONFIG_124M["context_length"]:
    print("Not enough tokens for the training loader. "
          "Try to lower the `GPT_CONFIG_124M['context_length']` or "
          "increase the `training_ratio`")

if total_tokens * (1-train_ratio) < GPT_CONFIG_124M["context_length"]:
    print("Not enough tokens for the validation loader. "
          "Try to lower the `GPT_CONFIG_124M['context_length']` or "
          "decrease the `training_ratio`")

In [ ]:
print("Train loader:")
for x, y in train_loader:
    print(x.shape, y.shape)

print("\nValidation loader:")
for x, y in val_loader:
    print(x.shape, y.shape)
    

In [ ]:
train_tokens = 0
for input_batch, target_batch in train_loader:
    train_tokens += input_batch.numel()

val_tokens = 0
for input_batch, target_batch in val_loader:
    val_tokens += input_batch.numel()

print("Training tokens:", train_tokens)
print("Validation tokens:", val_tokens)
print("All tokens:", train_tokens + val_tokens)

Given the input and targets, we forward pass through the model for the input batch. Then given the logits, we flatten on the dimensions and calculate the cross entropy loss.

In [ ]:
def calc_loss_batch(input_batch, target_batch, model):
    input_batch, target_batch = input_batch, target_batch
    logits = model(input_batch)
    loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())

    return loss


In [ ]:

def calc_loss_loader(data_loader, model, num_batches=None):
    total_loss = 0.
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        # Reduce the number of batches to match the total number of batches in the data loader
        # if num_batches exceeds the number of batches in the data loader
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(input_batch, target_batch, model)
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches

In [ ]:
torch.manual_seed(123) # For reproducibility due to the shuffling in the data loader

train_loss = calc_loss_loader(train_loader, model)
val_loss = calc_loss_loader(val_loader, model)

print("Training loss:", train_loss)
print("Validation loss:", val_loss)

## Training!!

In [ ]:
def train_model_simple(model, train_loader, val_loader, optimizer, num_epochs,
                       eval_freq, eval_iter, start_context, tokenizer):
    # Initialize lists to track losses and tokens seen
    train_losses, val_losses, track_tokens_seen = [], [], []
    tokens_seen, global_step = 0, -1

    # Main training loop
    for epoch in range(num_epochs):
        
        for input_batch, target_batch in train_loader:
            optimizer.zero_grad() # Reset loss gradients from previous batch iteration
            loss = calc_loss_batch(input_batch, target_batch, model)
            loss.backward() # Calculate loss gradients
            optimizer.step() # Update model weights using loss gradients
            tokens_seen += input_batch.numel()
            global_step += 1

            # Optional evaluation step
            if global_step % eval_freq == 0:
                train_loss, val_loss = evaluate_model(
                    model, train_loader, val_loader, eval_iter)
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                track_tokens_seen.append(tokens_seen)
                print(f"Ep {epoch+1} (Step {global_step:06d}): "
                      f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")

        # Print a sample text after each epoch
        generate_and_print_sample(
            model, tokenizer, start_context
        )

    return train_losses, val_losses, track_tokens_seen


def evaluate_model(model, train_loader, val_loader, eval_iter):
    train_loss = calc_loss_loader(train_loader, model, num_batches=eval_iter)
    val_loss = calc_loss_loader(val_loader, model, num_batches=eval_iter)
    return train_loss, val_loss


def generate_and_print_sample(model, tokenizer, start_context):
    context_size = model.pos_emb.weight.shape[0]
    encoded = text_to_token_ids(start_context, tokenizer)
    token_ids = generate_text_simple(
        model=model, idx=encoded,
        max_new_tokens=50, context_size=context_size
    )
    decoded_text = token_ids_to_text(token_ids, tokenizer)
    print(decoded_text.replace("\n", " "))  # Compact print format

In [ ]:
from scripts.perf_timer import PerfTimer

torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)

num_epochs = 10

timer = PerfTimer()

timer.start()
train_losses, val_losses, tokens_seen = train_model_simple(
    model, train_loader, val_loader, optimizer,
    num_epochs=num_epochs, eval_freq=5, eval_iter=5,
    start_context="Every effort moves you", tokenizer=tokenizer
)
timer.stop()

print(f"Took this long to train: {timer.elapsed_ms()} ms")


In [ ]:
import tiktoken
from scripts.generate import generate_text_simple

tokenizer = tiktoken.get_encoding("gpt2")

token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids("Every effort moves you", tokenizer),
    max_new_tokens=50,
    context_size=GPT_CONFIG_124M["context_length"]
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

## Saving the Model Weights

In [ ]:
torch.save(model.state_dict(), "models/gpt2-verdict-model.pth")

## Reload the Model

In [ ]:
import torch
from scripts.gpt2_model import GPTModel

model = GPTModel(GPT_CONFIG_124M)
model.load_state_dict(
  torch.load("models/gpt2-verdict-model.pth", weights_only=True)
)
model.eval()

In [ ]:
token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids("Every effort moves you", tokenizer),
    max_new_tokens=25,
    context_size=GPT_CONFIG_124M["context_length"]
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

## TTNN

We are not going to train the model using TTNN, instead, we will use it to perform inference in an already trained model. In this case, we diligently trained the GP2 model with our data set using CPU and had saved it to disk. We can reload those weights and reapply them to the GPTModel_ttnn class.


Redefine the config.

In [1]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,   # Vocabulary size
    "context_length": 256, # Shortened context length (orig: 1024)
    "emb_dim": 768,        # Embedding dimension
    "n_heads": 12,         # Number of attention heads
    "n_layers": 12,        # Number of layers
    "drop_rate": 0.0,      # Dropout rate
    "qkv_bias": False      # Query-key-value bias
}

Set some imports.

In [2]:
import ttnn
import tiktoken
import torch
from torch import nn
from scripts.text_helpers import text_to_token_ids, token_ids_to_text

torch.manual_seed(123)

device = None

2025-05-11 21:17:36.042 | DEBUG    | ttnn.library_tweaks:prepare_dir_as_metal_home:54 - Existing installation of 0.57.0rc60+any detected
2025-05-11 21:17:36.067 | DEBUG    | ttnn:<module>:83 - Initial ttnn.CONFIG:
Config{cache_path=/home/avgdev/.cache/ttnn,model_cache_path=/home/avgdev/.cache/ttnn/models,tmp_dir=/tmp/ttnn,enable_model_cache=false,enable_fast_runtime_mode=true,throw_exception_on_fallback=false,enable_logging=false,enable_graph_report=false,enable_detailed_buffer_report=false,enable_detailed_tensor_report=false,enable_comparison_mode=false,comparison_mode_should_raise_exception=false,comparison_mode_pcc=0.9999,root_report_path=generated/ttnn/reports,report_name=std::nullopt,std::nullopt}


## Open the Device

In [3]:
if device:
  ttnn.close_device(device)

device_id = 0
device = ttnn.open_device(device_id=device_id)

device.enable_program_cache()

                 Device | INFO     | Opening user mode device driver
2025-05-11 21:17:37.071 | INFO     | SiliconDriver   - Opened PCI device 0; KMD version: 1.33.0, IOMMU: disabled

2025-05-11 21:17:37.094 | INFO     | SiliconDriver   - Opened PCI device 0; KMD version: 1.33.0, IOMMU: disabled
2025-05-11 21:17:37.097 | INFO     | SiliconDriver   - Harvesting mask for chip 0 is 0x200 (physical layout: 0x1, logical: 0x200, simulated harvesting mask: 0x0).
2025-05-11 21:17:37.098 | INFO     | SiliconDriver   - Opened PCI device 0; KMD version: 1.33.0, IOMMU: disabled
2025-05-11 21:17:37.099 | INFO     | SiliconDriver   - Detected PCI devices: [0]
2025-05-11 21:17:37.099 | INFO     | SiliconDriver   - Using local chip ids: {0} and remote chip ids {}
2025-05-11 21:17:37.123 | [3

New chip! We now have 1 chips
Chip initialization complete (found )
Chip initializing complete...
 ARC

 [4/4] DRAM

 [16/16] ETH

 CPU

Chip detection complete (found )


8;2;100;149;237mINFO     | SiliconDriver   - Software version 6.0.0, Ethernet FW version 6.14.0 (Device 0)
                  Metal | INFO     | Initializing device 0. Program cache is NOT enabled
                  Metal | INFO     | AI CLK for device 0 is:   1000 MHz
                  Metal | INFO     | Enabling program cache on device 0


## Initialize the GPT Model

We will reload the model.

In [4]:
from scripts.gpt2_model import GPTModel
model = GPTModel(GPT_CONFIG_124M)
model.load_state_dict(
  torch.load("models/gpt2-verdict-model.pth", weights_only=True)
)
model.eval()

GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(256, 768)
  (drop_emb): Dropout(p=0.0, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=False)
        (W_key): Linear(in_features=768, out_features=768, bias=False)
        (W_value): Linear(in_features=768, out_features=768, bias=False)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (ff): FeedForward(
        (layer): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.0, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=

What we can do now is reload weights into our model_ttnn.

In [5]:
from scripts.gpt2_model_ttnn import GPTModel_ttnn, TransformerBlock_ttnn

model_ttnn = GPTModel_ttnn(GPT_CONFIG_124M, device)

model_ttnn.pos_emb.weight = torch.nn.Parameter(model.pos_emb.weight) 
model_ttnn.tok_emb.weight = torch.nn.Parameter(model.tok_emb.weight)

for i, block in enumerate(model.trf_blocks):
  t = model_ttnn.trf_blocks_ttnn[i]

  t.att.W_key.weight = torch.nn.Parameter(block.att.W_key.weight)
  t.att.W_key.bias = torch.nn.Parameter(block.att.W_key.bias)

  t.att.W_query.weight = torch.nn.Parameter(block.att.W_query.weight)
  t.att.W_query.bias = torch.nn.Parameter(block.att.W_query.bias)
  
  t.att.W_value.weight = torch.nn.Parameter(block.att.W_value.weight)
  t.att.W_value.bias = torch.nn.Parameter(block.att.W_value.bias)
  
  t.att.out_proj.weight = torch.nn.Parameter(block.att.out_proj.weight)
  t.att.out_proj.bias = torch.nn.Parameter(block.att.out_proj.bias)

  t.ff.lin_1.weight = torch.nn.Parameter(block.ff.layer[0].weight)
  t.ff.lin_1.bias = torch.nn.Parameter(block.ff.layer[0].bias)
  t.ff.lin_2.weight = torch.nn.Parameter(block.ff.layer[2].weight)
  t.ff.lin_2.bias = torch.nn.Parameter(block.ff.layer[2].bias)

  t.norm1.scale = torch.nn.Parameter(block.norm1.scale)
  t.norm1.shift = torch.nn.Parameter(block.norm1.shift)

  t.norm2.scale = torch.nn.Parameter(block.norm2.scale)
  t.norm2.shift = torch.nn.Parameter(block.norm2.shift)

model_ttnn.final_norm.shift = torch.nn.Parameter(model.final_norm.shift)
model_ttnn.final_norm.scale = torch.nn.Parameter(model.final_norm.scale)
model_ttnn.out_head.weight = torch.nn.Parameter(model.out_head.weight)

model_ttnn.update_weights()

In [6]:
model_ttnn.tok_emb_ttnn.shape

Shape([50257, 768])

## Compare Token Embeddings

In [7]:
from scripts.compare_tensors import compare_tensors

compare_tensors(
  ttnn.to_torch(
    ttnn.reshape(
      (model_ttnn.tok_emb_ttnn),
      (1, model_ttnn.tok_emb_ttnn.shape[0], model_ttnn.tok_emb_ttnn.shape[1])
    )
  ),
  model.tok_emb.weight.unsqueeze(0),
  suppress_details=True
)




Overall Status: ✅ PASS


{'max_diff': 0.0,
 'mean_diff': 0.0,
 'correlation': 1.0,
 'max_diff_status': True,
 'mean_diff_status': True,
 'correlation_status': True,
 'overall_status': True}

## Compare Positional Embeddings

In [8]:
from scripts.compare_tensors import compare_tensors

compare_tensors(
  ttnn.to_torch(
    ttnn.reshape(
      (model_ttnn.pos_emb_ttnn),
      (1, model_ttnn.pos_emb_ttnn.shape[0], model_ttnn.pos_emb_ttnn.shape[1])
    )
  ),
  model.pos_emb.weight.unsqueeze(0),
  suppress_details=True
)


Overall Status: ✅ PASS


{'max_diff': 0.0,
 'mean_diff': 0.0,
 'correlation': 0.99609375,
 'max_diff_status': True,
 'mean_diff_status': True,
 'correlation_status': True,
 'overall_status': True}

## Compare Final Norm

In [10]:
from scripts.compare_tensors import compare_tensors

final_norm_shift_shape = model_ttnn.final_norm.shift_ttnn.shape
final_norm_scale_shape = model_ttnn.final_norm.scale_ttnn.shape

compare_tensors(
  ttnn.to_torch(
    ttnn.reshape(
      model_ttnn.final_norm.shift_ttnn,
      (1, 1, final_norm_shift_shape[0])
    )
  ),
  model.final_norm.shift.unsqueeze(0).unsqueeze(0),
  suppress_details=True
)

print()

compare_tensors(
  ttnn.to_torch(
    ttnn.reshape(
      model_ttnn.final_norm.scale_ttnn,
      (1, 1, final_norm_scale_shape[0])
    )
  ),
  model.final_norm.scale.unsqueeze(0).unsqueeze(0),
  suppress_details=True
)


Overall Status: ✅ PASS


Overall Status: ✅ PASS


{'max_diff': 0.0,
 'mean_diff': 0.0,
 'correlation': 1.0078125,
 'max_diff_status': True,
 'mean_diff_status': True,
 'correlation_status': True,
 'overall_status': True}

## Out Head Comparison


In [11]:
from scripts.compare_tensors import compare_tensors

out_head_shape = model_ttnn.out_head_ttnn.shape
print(out_head_shape)

compare_tensors(
  ttnn.to_torch(
    ttnn.reshape(
      model_ttnn.out_head_ttnn,
      (1, out_head_shape[0], out_head_shape[1])
    )
  ),
  model.out_head.weight.unsqueeze(0),
  suppress_details=True
)



Shape([50257, 768])

Overall Status: ✅ PASS


{'max_diff': 0.0,
 'mean_diff': 0.0,
 'correlation': 1.0,
 'max_diff_status': True,
 'mean_diff_status': True,
 'correlation_status': True,
 'overall_status': True}

## Transformer Blocks Comparison

In [12]:
from scripts.compare_tensors import compare_tensors

for i, block in enumerate(model.trf_blocks):
  t = model_ttnn.trf_blocks_ttnn[i]

  compare_tensors(
    ttnn.to_torch(
      ttnn.reshape(
        t.att.W_key_ttnn,
        (1, t.att.W_key_ttnn.shape[0], t.att.W_key_ttnn.shape[1]))
    ),
    model.trf_blocks[i].att.W_key.weight.unsqueeze(0),
    suppress_details=True
  )
  compare_tensors(
    ttnn.to_torch(
      ttnn.reshape(
        t.att.W_query_ttnn,
        (1, t.att.W_query_ttnn.shape[0], t.att.W_query_ttnn.shape[1]))
    ),
    model.trf_blocks[i].att.W_query.weight.unsqueeze(0),
    suppress_details=True
  )
  compare_tensors(
    ttnn.to_torch(
      ttnn.reshape(
        t.att.W_value_ttnn,
        (1, t.att.W_value_ttnn.shape[0], t.att.W_value_ttnn.shape[1]))
    ),
    model.trf_blocks[i].att.W_value.weight.unsqueeze(0),
    suppress_details=True
  )
  compare_tensors(
    ttnn.to_torch(
      ttnn.reshape(
        t.att.out_proj_ttnn,
        (1, t.att.out_proj_ttnn.shape[0], t.att.out_proj_ttnn.shape[1]))
    ),
    model.trf_blocks[i].att.out_proj.weight.unsqueeze(0),
    suppress_details=True
  )
  compare_tensors(
    ttnn.to_torch(
      ttnn.reshape(
        t.att.out_proj_bias_ttnn,
        (1, 1, t.att.out_proj_bias_ttnn.shape[0]))
    ),
    model.trf_blocks[i].att.out_proj.bias.unsqueeze(0).unsqueeze(0),
    suppress_details=True
  )


Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status: ✅ PASS

Overall Status:

In [12]:
from scripts.generate_ttnn import generate_text_simple_ttnn

torch.manual_seed(123)
start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")

inputs_ttnn = ttnn.from_torch(
  text_to_token_ids(start_context, tokenizer),
  layout=ttnn.TILE_LAYOUT,
  dtype=ttnn.uint32,
  device=device
)
token_ids = generate_text_simple_ttnn(
  model=model_ttnn,
  idx_ttnn=inputs_ttnn,
  max_new_tokens=50,
  context_size=GPT_CONFIG_124M["context_length"],
  device=device
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Shape([1, 4])
                  Metal | WARNING  | Circular buffer indices are not contiguous starting at 0. This will hurt dispatch performance. Non-contiguous indices: 16. First unused index: 1. Kernels: reader_unary_pad_dims_split_rows_multicore
                  Metal | WARNING  | Circular buffer indices are not contiguous starting at 0. This will hurt dispatch performance. Non-contiguous indices: 16. First unused index: 1. Kernels: writer_unary_interleaved_start_id, reader_unary_pad_dims_split_rows_multicore, tilize
                  Metal | WARNING  | Circular buffer indices are not contiguous starting at 0. This will hurt dispatch performance. Non-contiguous indices: 2,3. First unused index: 1. Kernels: reader_unary_reduce_interleaved_start_id
                  Metal | WARNING  | Circular buffer indices are not contiguous starting at 0. This will hurt dispatch performance. Non-contiguous indices: 2,3. First unused index: 1. Kernels: writer_unary_interleaved_start_id, reader_unar

RuntimeError: TT_FATAL @ /work/ttnn/cpp/ttnn/operations/reduction/argmax/device/argmax_op.cpp:47: input_shape[0] == 1
info:
dim 0 must be 1
backtrace:
 --- /home/avgdev/code/ttnn-sandbox/.venv/lib/python3.10/site-packages/ttnn/_ttnn.cpython-310-x86_64-linux-gnu.so(+0xf2f318) [0x7e76b1d2f318]
 --- ttnn::operations::reduction::ArgMax::validate_with_output_tensors(std::__1::vector<tt::tt_metal::Tensor, std::__1::allocator<tt::tt_metal::Tensor>> const&, std::__1::vector<std::__1::optional<tt::tt_metal::Tensor>, std::__1::allocator<std::__1::optional<tt::tt_metal::Tensor>>> const&) const
 --- /home/avgdev/code/ttnn-sandbox/.venv/lib/python3.10/site-packages/ttnn/_ttnn.cpython-310-x86_64-linux-gnu.so(_ZN4ttnn16device_operation6detail23launch_on_worker_threadIN2tt8tt_metal9operation23OldInfraDeviceOperationINSt3__16vectorINS4_6TensorENS7_9allocatorIS9_EEEEEENS3_3stl10StrongTypeIhNS_10QueueIdTagEEElNS5_15DeviceOperationISC_EENSD_13tensor_args_tESC_PNS4_7IDeviceEEEvT0_T1_RKT2_RKT3_RT4_RT5_+0x23

In [ ]:
ttnn.close_device(device)

In [13]:
ttnn.argmax(
  ttnn.from_torch(
    torch.tensor([[1, 3, 2]]), 
    dtype=ttnn.bfloat16,
    layout=ttnn.ROW_MAJOR_LAYOUT,
    device=device
  ),
  dim=-1
)

RuntimeError: TT_THROW @ /work/tt_metal/common/shape_base.cpp:29: tt::exception
info:
ShapeBase[] index out of range. 2 not in [-4, 2)
backtrace:
 --- /home/avgdev/code/ttnn-sandbox/.venv/lib/python3.10/site-packages/ttnn/_ttnn.cpython-310-x86_64-linux-gnu.so(+0x1a2668b) [0x724f1282668b]
 --- tt::tt_metal::ShapeBase::operator[](int)
 --- ttnn::operations::reduction::ArgMax::compute_output_specs(std::__1::vector<tt::tt_metal::Tensor, std::__1::allocator<tt::tt_metal::Tensor>> const&, std::__1::vector<std::__1::optional<tt::tt_metal::Tensor>, std::__1::allocator<std::__1::optional<tt::tt_metal::Tensor>>> const&) const
 --- ttnn::operations::reduction::ArgMax::create_output_tensors(std::__1::vector<tt::tt_metal::Tensor, std::__1::allocator<tt::tt_metal::Tensor>> const&, std::__1::vector<std::__1::optional<tt::tt_metal::Tensor>, std::__1::allocator<std::__1::optional<tt::tt_metal::Tensor>>> const&) const
 --- /home/avgdev/code/ttnn-sandbox/.venv/lib/python3.10/site-packages/ttnn/_ttnn.cpyt